## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,35.76,70,100,21.14,overcast clouds
1,1,Broome,US,42.2506,-75.8330,45.63,48,1,4.00,clear sky
2,2,Saldanha,ZA,-33.0117,17.9442,64.45,92,100,13.73,overcast clouds
3,3,Lisakovsk,KZ,52.5369,62.4936,4.57,94,8,4.54,clear sky
4,4,Manzhouli,CN,49.6000,117.4333,6.71,86,100,4.74,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,East London,ZA,-33.0153,27.9116,71.98,94,100,3.87,moderate rain
6,6,Vaini,TO,-21.2000,-175.2000,77.16,94,75,8.05,light rain
8,8,Mahebourg,MU,-20.4081,57.7000,79.09,69,20,8.05,few clouds
11,11,Praia,CV,14.9215,-23.5087,75.74,69,98,11.50,overcast clouds
18,18,Santa Vitoria Do Palmar,BR,-33.5189,-53.3681,76.68,59,0,14.90,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

221


City_ID                221
City                   221
Country                217
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

217


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,East London,ZA,-33.0153,27.9116,71.98,94,100,3.87,moderate rain
6,6,Vaini,TO,-21.2000,-175.2000,77.16,94,75,8.05,light rain
8,8,Mahebourg,MU,-20.4081,57.7000,79.09,69,20,8.05,few clouds
11,11,Praia,CV,14.9215,-23.5087,75.74,69,98,11.50,overcast clouds
18,18,Santa Vitoria Do Palmar,BR,-33.5189,-53.3681,76.68,59,0,14.90,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,East London,ZA,71.98,moderate rain,-33.0153,27.9116,
6,Vaini,TO,77.16,light rain,-21.2000,-175.2000,
8,Mahebourg,MU,79.09,few clouds,-20.4081,57.7000,
11,Praia,CV,75.74,overcast clouds,14.9215,-23.5087,
18,Santa Vitoria Do Palmar,BR,76.68,clear sky,-33.5189,-53.3681,
22,Rikitea,PF,77.40,light rain,-23.1203,-134.9692,
28,Atuona,PF,79.16,few clouds,-9.8000,-139.0333,
32,Hilo,US,78.28,few clouds,19.7297,-155.0900,
35,George Town,MY,80.89,scattered clouds,5.4112,100.3354,
39,Bandarbeyla,SO,76.86,clear sky,9.4942,50.8122,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,East London,ZA,71.98,moderate rain,-33.0153,27.9116,Tu Casa
6,Vaini,TO,77.16,light rain,-21.2000,-175.2000,Keleti Beach Resort
8,Mahebourg,MU,79.09,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
11,Praia,CV,75.74,overcast clouds,14.9215,-23.5087,LT Aparthotel
18,Santa Vitoria Do Palmar,BR,76.68,clear sky,-33.5189,-53.3681,Hostel Comodoro
22,Rikitea,PF,77.40,light rain,-23.1203,-134.9692,People ThankYou
28,Atuona,PF,79.16,few clouds,-9.8000,-139.0333,Villa Enata
32,Hilo,US,78.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
35,George Town,MY,80.89,scattered clouds,5.4112,100.3354,Cititel Penang
39,Bandarbeyla,SO,76.86,clear sky,9.4942,50.8122,JABIR HOTEL


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))